In [3]:
import math
import requests
from bs4 import BeautifulSoup
import json
import time
import random
import numpy as np
import re

%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv

load_dotenv()
NGSESSID = os.getenv("NGSESSID")


cookies = {
    'NGSESSID': NGSESSID,
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 11.0; Win64; x64; rv:121.0) Gecko/20100100 Firefox/121.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US;q=0.5,en;q=0.3',
    'Referer': 'https://www.connexservice.com/',
    'X-Requested-With': 'XMLHttpRequest',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
}

# response = requests.get(
#     'https://www.connexservice.com/HolidayPlus_Preview/search?filters=%7B%22travel_period%22:%7B%22from%22:null,%22to%22:null%7D,%22travel_themes%22:[],%22fittings%22:[],%22positions%22:[],%22countries%22:null,%22regions%22:null%7D&paging=%7B%22limit%22:15,%22offset%22:15%7D',
#     cookies=cookies,
#     headers=headers,
# )

In [16]:
amount = 50
offset = 0
total = 1253
hotel_ids = []

# load all hotels, random sleep to avoid being blocked

while offset < total:
    if offset % 100 == 0:
        print(offset / total * 100, '%')
    with requests.session() as session:
        response = requests.get(
            f'https://www.connexservice.com/HolidayPlus_Preview/search?filters=%7B%22travel_period%22:%7B%22from%22:null,%22to%22:null%7D,%22travel_themes%22:[],%22fittings%22:[],%22positions%22:[],%22countries%22:null,%22regions%22:null%7D&paging=%7B%22limit%22:{amount},%22offset%22:{offset}%7D',
            cookies=cookies,
            headers=headers,
        )        
        
        # with open('hotel.html', 'wb+') as f:
        #     f.write(response.content)
        # 
        # print(response.content)

        data = json.loads(response.content)
        keys = 'name', 'country_message_key', 'country_region_message_key', 'prm_partner_id', 'location'
        keys_new = 'name', 'country', 'region', 'id', 'location'
        for hotel in data['results']['partners']['result']:
            new_hotel = {}
            for key, key_new in zip(keys, keys_new):
                if key not in hotel['doc'].keys():
                    print(hotel)
                    print('missing key', key)
                    print('\n')
                else:
                    new_hotel[key_new] = hotel['doc'][key]
            hotel_ids.append(new_hotel)
        offset += amount
        time.sleep(random.uniform(4.5, 9.5))
        
    # break

0.0 %
7.980845969672785 %
15.96169193934557 %
23.942537909018355 %
31.92338387869114 %
{'doc': {'prm_partner_id': '36489', 'city': 'Castellón', 'hotel_id': 'H0041327', 'country_message_key': 'Spain', 'partner_number': 'H0041327', 'region_id': '528', 'travel_themes': ['38'], 'language': '11', 'positions': ['55'], 'logo_src': '', 'img_src': 'https://imgs.connexservice.com/partner_img/H0042000/H0041327/H0041327_ag.jpg', 'zipcode': '12594', 'classifications': [{'cnt_total': '12', 'max_classification': '5', 'id': '6', 'categories': [], 'classification': None, 'cnt_matches': '0'}], 'is_freedreams_hotel': '1', 'phone': '+43724220256031', 'street': 'Carrer dels Terrers, s/n', 'name': 'Hotel Gran Duque', 'fittings': ['40', '41', '43', '44', '45', '48', '51', '52', '53'], 'currency': '1', 'fax': None, 'country_region_message_key': 'Valencian Community', 'email': 'booking@hotelvoucher.eu', 'country_iso': 'ES', 'homepage': 'https://www.marinador.com/es/hoteles-apartamentos/hotel-gran-duque-4#infor

In [10]:
data = json.loads(response.content)
for hotel in data['results']['partners']['result']:
    # print(hotel['doc']['prm_partner_id'])
    # hotel_ids.append([hotel['doc']['name'], hotel['doc']['country_message_key'], hotel['doc']['country_region_message_key'], hotel['doc']['prm_partner_id']])
    hotel_ids.append({'name': hotel['doc']['name'], 'country': hotel['doc']['country_message_key'], 'region': hotel['doc']['country_region_message_key'], 'id': hotel['doc']['prm_partner_id'], 'location': hotel['doc']['location']}) 



In [185]:
temp = data['results']['partners']['result']
print(temp)


[{'doc': {'prm_partner_id': '2004', 'city': 'Prag/Praha', 'hotel_id': 'H0024907', 'country_message_key': 'Česká republika', 'partner_number': 'H0024907', 'region_id': '423', 'travel_themes': ['38'], 'language': '89', 'positions': [], 'logo_src': '', 'img_src': 'https://imgs.connexservice.com/partner_img/H0026000/H0024907/H0024907_ag.jpg', 'zipcode': '10000', 'classifications': [{'cnt_total': '12', 'max_classification': '5', 'id': '6', 'categories': ['438', '449', '455', '471', '500'], 'classification': '2', 'cnt_matches': '5'}], 'phone': '00420 274 777 717', 'street': 'Okrajní 1', 'name': 'Hotel Villa', 'fittings': [], 'currency': '1', 'location': {'lon': 14.48241, 'lat': 50.081577}, 'fax': '00420 274 777 719', 'country_region_message_key': 'Praha', 'email': 'reservation@hotel-villa.cz', 'country_iso': 'CZ', 'homepage': 'http://www.hotel-villa.cz', 'text_short': 'V klidném místě vilové čtvrti Prahy, blízko stanice metra Želivského, Vás očekává hotel Villa, který je obklopen překrásnou 

In [190]:
for i, hotel in enumerate(hotel_ids):
    # print(i, hotel[3], temp[i]['doc']['prm_partner_id'])
    if len(hotel) > 6:
        continue
    for j in temp:
        if hotel[3] == j['doc']['prm_partner_id']:
            # print(j)
            if 'location' in j['doc'].keys():
                hotel.append(j['doc']['location'])
            else:
                print('no location for')
                print(hotel)
            hotel.append(j['doc']['homepage'])
            break
    else:
        print(hotel)

In [18]:
counter = 0
with requests.session() as session:
    for hotel in hotel_ids:
        if 'text' in hotel.keys():
            counter += 1
            continue
        if counter % 10 == 0:
            print(counter / len(hotel_ids) * 100, '%')
        counter += 1
        
        response1 = requests.get('https://www.connexservice.com/HolidayPlus_Preview/hoteldetail?partner_id='+hotel['id'], cookies=cookies, headers=headers,)
        # with open('hotel1.html', 'wb+') as f:
        #     f.write(response1.content)
        soup = BeautifulSoup(response1.content, "html.parser")
        bar = soup.find("div", {"class": "col-xs-12 padding-0-r padding-0-l"})
        if bar is None:
            print('no text for')
            print(hotel)
            continue
        text = re.sub('ww+', '', bar.text)
        text = re.sub(r'\s{2,}', '\n', text)
        hotel['text'] = text
        
        time.sleep(random.uniform(4.5, 9.5))

        # break


0.0 %
no text for
{'name': 'Hotel Christoph', 'country': 'Italy', 'region': 'South Tyrol', 'id': '2147', 'location': {'lon': 12.011268, 'lat': 46.748245}}
0.8 %
1.6 %
2.4 %
3.2 %
4.0 %
4.8 %
5.6000000000000005 %
6.4 %
7.199999999999999 %
8.0 %
8.799999999999999 %
9.6 %
10.4 %
11.200000000000001 %
12.0 %
12.8 %
13.600000000000001 %
14.399999999999999 %
15.2 %
16.0 %
16.8 %
17.599999999999998 %
18.4 %
19.2 %
20.0 %
20.8 %
21.6 %
22.400000000000002 %
23.200000000000003 %
24.0 %
24.8 %
25.6 %
26.400000000000002 %
27.200000000000003 %
28.000000000000004 %


In [71]:
# save data

with open('hotel_ids.json', 'w') as f:
    json.dump(hotel_ids, f)

In [4]:
# load data

with open('hotel_ids.json', 'r') as f:
    hotel_ids = json.load(f)

In [43]:
with open('hotel.html', 'wb+') as f:
    f.write(response1.content)


In [12]:
for hotel in hotel_ids:
    print(hotel['name'])
    print(hotel['text'])
    print('\n')


Hotel Villa

Stay with TRAVELCARD unlimited
:
1-6 Night(s) in a double room for 2 people
Breakfast and dinner per person per night are mandatory.
Price per person per night for breakfast and dinner:
EUR 48.00
per person
per night
Dinner consists of a 3-course à la carte menu



Le Palais Art Hotel Prague

Stay with TRAVELCARD unlimited
:
1-6 Night(s) in a double room for 2 people
Breakfast and dinner per person per night are mandatory.
Price per person per night for breakfast and dinner:
EUR 80.00
per person
per night
Includes 1 glass of prosecco Due to the current opening times of the restaurant from Monday to Friday 11:00 a.m. to 4:00 p.m., lunch is compulsory instead of dinner.



Hotel Ametyst

Stay with TRAVELCARD unlimited
:
1-6 Night(s) in a double room for 2 people
Breakfast and dinner per person per night are mandatory.
Price per person per night for breakfast and dinner:
01.01. - 31.03.:
EUR 50.00
per person
per night
Accomodation in room category Deluxe
Dinner consists of th

In [5]:
text1 = '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nEUR\xa08,50\nna osobu\nza noc\nRate per person and night for the à la carte menu\nEUR\xa025,00\nna osobu\nza noc\n'

text2 = 'Stay with TRAVELCARD unlimited\n:\n1-6 Night(s) in a double room for 2 people\nBreakfast and dinner per person per night are mandatory.\nRate per person and night for breakfast:\nEUR 9.00\nper person\nper night\nRate per person and night for the 3-course menu:\nEUR 16.00\nper person\nper night'

text3 = '\nStay with TRAVELCARD unlimited\n:\n1-6 Night(s) in a double room for 2 people\nBreakfast and dinner per person per night are mandatory.\nRate per person and night for breakfast:\nsummer:\nEUR 14.00\nper person\nper night\nwinter:\nEUR 14.00\nper person\nper night\nRat'

# text = hotel_ids[69][4]
# print(text)


def price(text):
    lines = text.split("\n")

    summer_breakfast_price = 0
    winter_breakfast_price = 0
    summer_menu_price = 0
    winter_menu_price = 0

    currencies = ["EUR", "GBP", "CHF"]

    for i, line in enumerate(lines):
        for currency in currencies:
            brk = True
            if "Rate per person and night for breakfast:" in line and i+1 < len(lines):
                summer_breakfast_price = float(lines[i+1].split(" ")[1].replace(",", "."))
            elif "Rate per person and night for the 3-course menu:" in line and i+1 < len(lines):
                summer_menu_price = float(lines[i+1].split(" ")[1].replace(",", "."))
            elif "Cena za snídaní " in line or " za snídani " in line or "breakfast and dinner" in line and i+2 < len(lines):
                if "léto: "+currency in lines[i+1]:
                    summer_breakfast_price = float(lines[i+1].split(" ")[2].replace(",", "."))
                elif "zima: "+currency in lines[i+2]:
                    winter_breakfast_price = float(lines[i+2].split(" ")[2].replace(",", "."))
                elif currency in lines[i+1]:
                    # print(lines[i+1].split("\xa0"))
                    winter_breakfast_price = float(lines[i+1].split("\xa0")[1].replace(",", "."))
                elif currency in lines[i+2]:
                    # print(lines[i+1].split("\xa0"))
                    winter_breakfast_price = float(lines[i+2].split("\xa0")[1].replace(",", "."))
                else:
                    brk = False
            elif ((("menu" in line or "večeř" in line) and ":" in line) or "per person and night for" in line) and i+2 < len(lines):
                if "léto: "+currency in lines[i+1]:
                    summer_menu_price = float(lines[i+1].split(" ")[2].replace(",", "."))
                elif "zima: "+currency in lines[i+2]:
                    winter_menu_price = float(lines[i+2].split(" ")[2].replace(",", "."))
                elif currency in lines[i+1]:
                    # print(lines[i+1].split("\xa0"))
                    winter_menu_price = float(lines[i+1].split("\xa0")[1].replace(",", "."))
                elif currency in lines[i+2]:
                    # print(lines[i+1].split("\xa0"))
                    winter_menu_price = float(lines[i+2].split("\xa0")[1].replace(",", "."))
                else:
                    brk = False
            if brk:
                break

    min_summer_price = summer_breakfast_price + summer_menu_price
    min_winter_price = winter_breakfast_price + winter_menu_price

    # print("Summer:", min_summer_price)
    # print("Winter:", min_winter_price)

    return min(min_summer_price, min_winter_price) if min_summer_price != 0 and min_winter_price != 0 else max(min_summer_price, min_winter_price)

print(price(text1))
print(price(text2))
print(price(text3))


33.5
25.0


IndexError: list index out of range

In [60]:
import re

start = '(?:Rate|Price).*?'
end = '.*?^(?:From )?([a-z]+)\s*([\d.,]+)'

def get_hotel_price(text):
    price = 0
    prices = re.findall(start+r'breakfast and dinner'+end, text, re.MULTILINE | re.IGNORECASE | re.DOTALL | re.UNICODE)
    if prices:
        currency, prices = zip(*prices)
        print(currency, prices)
        prices = [price.replace(',', '.') for price in prices]
        prices = np.array(prices, dtype=float)
        price = min(price for price in prices)
    # Extracting meal prices
    else:
        breakfast_prices = re.findall(start+r'breakfast'+end, text, re.MULTILINE | re.IGNORECASE | re.DOTALL | re.UNICODE)
        dinner_prices = re.findall(start+r'(?:menu|dinner|evening meal)'+end, text, re.MULTILINE | re.IGNORECASE | re.DOTALL | re.UNICODE)
        
        if len(breakfast_prices) == 0 and len(dinner_prices) == 0:
            prices = re.findall(end, text, re.MULTILINE | re.IGNORECASE | re.DOTALL | re.UNICODE)
            currency, prices = zip(*prices)
            if len(prices) == 0:
                print('No prices found')
                return 0
            if len(prices) > 1:
                print('Many weird prices found')
            prices = [price.replace(',', '.') for price in prices]
            prices = np.array(prices, dtype=float)
            price = min(price for price in prices)
            return price
        
        currency1, breakfast_prices = zip(*breakfast_prices)
        
        if len(breakfast_prices) == 0 or len(dinner_prices) == 0:
            print(breakfast_prices)
            price = min(np.array(breakfast_prices, dtype=float))
            return price
        
        breakfast_prices = [price.replace(',', '.') for price in breakfast_prices]
        breakfast_price = min(np.array(breakfast_prices, dtype=float))
        currency2, dinner_prices = zip(*dinner_prices)
        
        if currency1 != currency2:
            print('Different currencies for breakfast and dinner')
            print(currency1, currency2)
        # to int
        dinner_prices = [price.replace(',', '.') for price in dinner_prices]
        dinner_price = min(np.array(dinner_prices, dtype=float))
        print(currency1, breakfast_prices, dinner_prices)

            
        price = breakfast_price + dinner_price

    return price

# Test cases
text1 = 'Stay with TRAVELCARD unlimited\n:\n1-6 Night(s) in a double room for 2 people\nBreakfast and dinner per person per night are mandatory.\nPrice per person per night for breakfast and dinner:\nEUR 48.00\nper person\nper night\nDinner consists of a 3-course à la carte menu'
text2 = 'Stay with TRAVELCARD unlimited\n:\n1-6 Night(s) in a double room for 2 people\nBreakfast and dinner per person per night are mandatory.\nRate per person and night for breakfast:\nEUR 9.00\nper person\nper night\nRate per person and night for the 3-course menu:\nEUR 16.00\nper person\nper night'
text3 = '\nStay with TRAVELCARD unlimited\n:\n1-6 Night(s) in a double room for 2 people\nBreakfast and dinner per person per night are mandatory.\nRate per person and night for breakfast:\nsummer:\nEUR 14.00\nper person\nper night\nwinter:\nEUR 14.00\nper person\nper night\nRat'

print("Price for text1:", get_hotel_price(text1))
print("Price for text2:", get_hotel_price(text2))
# print("Price for text3:", get_hotel_price(text3))


('EUR',) ('48.00',)
Price for text1: 48.0
('EUR',) ['9.00'] ['16.00']
Price for text2: 25.0


In [61]:
i = 0

for hotel in hotel_ids:
    if i % 10 == 0:
        print(i / len(hotel_ids) * 100, '%')
    if 'text' not in hotel.keys():
        hotel['cost'] = -1
        print(hotel)
    else:
        # print(hotel['name'])
        # print(hotel['text'])
        cost = get_hotel_price(hotel['text'])
        if cost == 0:
            print(hotel['text'])
            print('PANIK')
        hotel['cost'] = cost


0.0 %
('EUR',) ('48.00',)
0.0 %
('EUR',) ('80.00',)
0.0 %
('EUR',) ('50.00',)
0.0 %
('EUR',) ('31.00',)
0.0 %
('EUR',) ['12.00'] ['30.00']
0.0 %
('EUR',) ['15.00'] ['40.00']
0.0 %
('EUR',) ('65.00',)
0.0 %
('EUR',) ('35.00',)
0.0 %
('EUR',) ('47.00',)
0.0 %
('EUR',) ('49.00',)
0.0 %
('EUR',) ('60.00',)
0.0 %
('EUR',) ('35.00',)
0.0 %
('EUR',) ('35.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('42.00',)
0.0 %
('EUR',) ('59.00',)
0.0 %
('EUR',) ('60.00',)
0.0 %
('EUR',) ['15.00'] ['26.00']
0.0 %
('EUR',) ('60.00',)
0.0 %
('EUR',) ('60.00',)
0.0 %
('EUR',) ('95.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('98.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('54.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('45.00',)
0.0 %
('EUR',) ('60.00',)
0.0 %
('EUR',) ('49.00',)
0.0 %
('EUR',) ('40.00',)
0.0 %
('EUR',) ('45.00',)
0.0 %
('EUR',) ('45.00',)
0.0 %
('EUR',) ('35.00',)
0.0 %
('EUR',) ('50.00',)
0.0 %
('EUR

In [70]:
import folium

# Create a map centered on the mean latitude and longitude of the data points
mean_lat = 50.0755
mean_lon = 14.4378
map = folium.Map(location=[mean_lat, mean_lon], zoom_start=3)

def get_icon(price):
    # Define base icon size
    base_icon_size = 200
    
    icon_size = base_icon_size - price
    icon_size = max(0, icon_size/10)
    
    # Create and return the icon object
    return folium.Icon(icon='house', icon_size=(icon_size, icon_size))


# Add a marker for each data point
for hotel in hotel_ids:
    if 'location' not in hotel.keys():
        l = {'lat': mean_lat, 'lon': mean_lon}
    else:
        l = hotel['location']
    pop = ''
    for key, value in hotel.items():
        if key != 'location':
            pop += f'{key}: {value}     \n'
    price = hotel['cost']
    
    folium.Marker(
        location=[l['lat'], l['lon']],
        popup=pop,
        icon=get_icon(price)
    ).add_to(map)
    # break

# Save the map to an HTML file
map.save('map.html')


In [63]:
# order by price

hotel_ids.sort(key=lambda x: x['cost'])
for hotel in hotel_ids:
    print(hotel['name'], hotel['cost'])
    if 'text' in hotel.keys():
        print(hotel['text'])
    print('\n')

Hotel Christoph -1


Dworek Wymysłowo 16.0

Stay with TRAVELCARD unlimited
:
1-6 Night(s) in a double room for 2 people
Breakfast and dinner per person per night are mandatory.
Price per person per night for breakfast and dinner:
EUR 16.00
per person
per night
Dinner consists of a 3-course-menu.



Hotel Kastiel 16.0

Stay with TRAVELCARD unlimited
:
1-6 Night(s) in a double room for 2 people
Breakfast and dinner per person per night are mandatory.
Price per person per night for breakfast and dinner:
EUR 16.00
per person
per night
Dinner consists of a 3-course-menu



Hotel Azalia 18.0

Stay with TRAVELCARD unlimited
:
1-6 Night(s) in a double room for 2 people
Breakfast and dinner per person per night are mandatory.
Price per person per night for breakfast and dinner:
EUR 18.00
per person
per night
Dinner consists of a 3-course-menu



Hotel Swieradów 18.0

Stay with TRAVELCARD unlimited
:
1-6 Night(s) in a double room for 2 people
Breakfast and dinner per person per night are mandato